In [ ]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing import image
import tensorflow as tf


In [ ]:

def predict_image(model_path, image_path):
    # Extract model name from the filename
    model_name = os.path.basename(model_path).split('_')[0]
    
    # Set image size based on model
    img_h, img_w = (299, 299) if model_name == 'Xception' else (224, 224)
    
    # Map model name to preprocessing function
    model_config = {
        'VGG19': tf.keras.applications.vgg19.preprocess_input,
        'ResNet50': tf.keras.applications.resnet50.preprocess_input,
        'VGG16': tf.keras.applications.vgg16.preprocess_input,
        'MobileNetV2': tf.keras.applications.mobilenet_v2.preprocess_input,
        'Xception': tf.keras.applications.xception.preprocess_input,
        'EfficientNetB0': tf.keras.applications.efficientnet.preprocess_input,
        'DenseNet121': tf.keras.applications.densenet.preprocess_input
    }
    
    preprocess_input = model_config[model_name]
    
    # Load the entire model directly
    model = tf.keras.models.load_model(model_path)
    
    # Load and preprocess image
    img = image.load_img(image_path, target_size=(img_h, img_w))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_preprocessed = preprocess_input(img_array)
    
    # Direct prediction using the full model
    prediction = model.predict(img_preprocessed, verbose=0)
    
    # Format results
    img_name = os.path.basename(image_path)
    return (model_name, img_name, 'overriding' if prediction[0][0] > 0.5 else 'oblique')



In [ ]:

# Paths configuration
Base_Folder = 'D:/Learning/University of sadat/Grade 4/Semester 2/06- Graduation Project/Coding/'
Models_folder = f"{Base_Folder}05- Saved Models/"
Images_Folder = f"{Base_Folder}Test Data/"

# Collect model paths
models = [f"{Models_folder}{file}" for file in os.listdir(Models_folder) if file.lower().endswith(".h5")]

# Generate predictions
Results = []
for img in os.listdir(Images_Folder):
    for model in models:
        Results.append(predict_image(model, f"{Images_Folder}{img}"))

# Print results
for model_name, img_name, answer in Results:
    print(f"The prediction result of the \033[91m{model_name}\033[0m model for image {img_name} is \033[92m{answer}\033[0m class")